In [1]:
import altair as alt
import pandas as pd
import numpy as np


In [2]:
import os
fn = os.path.join(
    os.getcwd(),
    r'airbnb_info/allListingsNovember.xlsx'
)


bnbdf_twodays = pd.read_excel(
    fn,
    sheet_name = 'two_day_bookings_All'
)

bnbdf_weekly = pd.read_excel(
    fn,
    sheet_name = 'Weekly_bookings_All'
)


In [3]:
bnbdf_twodays = bnbdf_twodays.assign(
    **{
        'cost_nofees': bnbdf_twodays.loc[:, 'structuredStayDisplayPrice.primaryLine.accessibilityLabel'].apply(
            lambda x:
            int(
                ''.join(
                    filter(
                        str.isdigit, x.split()[0]
                    )
                )
            )
        ),
        'total_cost': bnbdf_twodays.loc[:, 'costBreakdown'].apply(
            lambda x:
            sum(
                [
                int(
                    ''.join(
                        filter(
                            str.isdigit, i
                        )
                    )
                ) for i in x.split(':')
            ][1:]
            )
        )
    }
)

bnbdf_weekly = bnbdf_weekly.assign(
    **{
        'cost_nofees': bnbdf_weekly.loc[:, 'structuredStayDisplayPrice.primaryLine.accessibilityLabel'].apply(
            lambda x:
            int(
                ''.join(
                    filter(
                        str.isdigit, x.split()[0]
                    )
                )
            )
        ),
        'total_cost': bnbdf_weekly.loc[:, 'costBreakdown'].apply(
            lambda x:
            sum(
                [
                int(
                    ''.join(
                        filter(
                            str.isdigit, i
                        )
                    )
                ) for i in x.split(':')
            ][1:]
            )
        )
    }
)

In [4]:
underfive_twodays = bnbdf_twodays[
    bnbdf_twodays.distanceFromTarryallCabin.apply(
        lambda x: 
        float(x.split()[0]) < 3.9
    )
]
underfive_weekly = bnbdf_weekly[
    bnbdf_weekly.distanceFromTarryallCabin.apply(
        lambda x: 
        float(x.split()[0]) < 3.9
    )
]



In [5]:
underfive_weekly[
    underfive_weekly.name == underfive_weekly.name.unique()[-1]
].drop_duplicates()

,name,distanceFromTarryallCabin,avgRating,reviewsCount,isNewListing,isSuperhost,amenities,structuredStayDisplayPrice.primaryLine.accessibilityLabel,costBreakdown,title,personCapacity,availableBeds,start_date,timeBooked,distanceFromTarryallCabin_w,cost_nofees,total_cost
83,Beautiful Log Cabin High In Colorado Mountains,3.823253281662541 miles,4.85,145,False,True,Wifi | Kitchen | Free parking,$128 per night,$135 x 7 nights: $943 | Weekly discount: -$47 ...,Cabin in Jefferson,6,4 beds,2022-11-01,2022-11-01:2022-11-08,3.823253,128,1253
84,Beautiful Log Cabin High In Colorado Mountains,3.823253281662541 miles,4.85,145,False,True,Wifi | Kitchen | Free parking,$128 per night,$135 x 7 nights: $943 | Weekly discount: -$47 ...,Cabin in Jefferson,6,4 beds,2022-11-08,2022-11-08:2022-11-15,3.823253,128,1253
85,Beautiful Log Cabin High In Colorado Mountains,3.823253281662541 miles,4.85,145,False,True,Wifi | Kitchen | Free parking,$204 per night,"$215 x 7 nights: $1,503 | Weekly discount: -$7...",Cabin in Jefferson,6,4 beds,2022-11-22,2022-11-22:2022-11-29,3.823253,204,1917


In [6]:
underfive_weekly.start_date.unique()

array(['2022-11-15T00:00:00.000000000', '2022-11-22T00:00:00.000000000',
       '2022-11-01T00:00:00.000000000', '2022-11-08T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [7]:
alt.Chart(
    underfive_weekly,
    title = 'Weekly Booking Costs for AirBnBs Closest 10: Total Cost'
).mark_line().encode(
    x='start_date',
    y='total_cost',
    color='name',
    strokeDash='name',
)

alt.Chart(...)

In [8]:
charts = (
    alt.Chart(
        underfive_weekly,
        title = 'Weekly Booking Costs for AirBnBs Closest 10: Total Cost'
    ).mark_line().encode(
        x='start_date',
        y='total_cost',
        color='name',
        strokeDash='name',
    ), alt.Chart(
        underfive_weekly,
        title = 'Weekly Booking Costs for AirBnBs Closest 10: Starting Cost'
    ).mark_line().encode(
        x='start_date',
        y='cost_nofees',
        color='name',
        strokeDash='name',
    ), alt.Chart(
        underfive_twodays,
        title = '2-Day Booking Costs for AirBnBs Closest 10: Total Cost'
    ).mark_line().encode(
        x='start_date',
        y='total_cost',
        color='name',
        strokeDash='name',
    ), alt.Chart(
        underfive_twodays,
        title = '2-Day Booking Costs for AirBnBs Closest 10: Starting Cost'
    ).mark_line().encode(
        x='start_date',
        y='cost_nofees',
        color='name',
        strokeDash='name',
    )
)

In [9]:
charts[2]

alt.Chart(...)

In [10]:
tuple(
    map(
        lambda chart:
        chart.save(
            os.path.join(
                os.getcwd(),
                fr'airbnb_info/{chart.title}.html'
            ),
            scale_factor = 2
        ),
        charts
    )
)

(None, None, None, None)